# Weather data in GEE

## Initializing dependencies

In [3]:
# Import necessary libraries
import ee  # Google Earth Engine API
import geemap  # For interactive maps with GEE and other utilities
import pandas as pd  # For data manipulation
import numpy as np  # For numerical operations
import matplotlib.pyplot as plt  # For plotting
import geopandas as gpd  # For geospatial data handling

In [5]:
ee.Authenticate()


Successfully saved authorization token.


In [6]:
ee.Initialize()

## TerraClimate

In [ ]:
import ee
import geemap
import geopandas as gpd
import pandas as pd

# Initialize Google Earth Engine
ee.Initialize()

# Load TerraClimate dataset
terraclimate = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE") \
    .filterDate("2000-01-01", "2023-12-31") \
    .select(["pdsi", "pr", "tmmn", "tmmx"])

# Load AMC shapefile
amc_shapefile_path = "../Variables/amc_1991_2022/amc_1991_2022.shp"
amc_gdf = gpd.read_file(amc_shapefile_path)
amc_fc = geemap.geopandas_to_ee(amc_gdf)

# Function to calculate mean of variables (weighted approach)
def calculate_yearly_mean(year):
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"
    
    # Filter TerraClimate for the year
    yearly_data = terraclimate.filterDate(start_date, end_date).mean()
    
    # Reduce to mean for each variable
    stats = yearly_data.reduceRegions(
        collection=amc_fc,
        reducer=ee.Reducer.mean(),
        scale=4638,
        crs="EPSG:4326"
    )
    
    # Convert to pandas DataFrame
    df = geemap.ee_to_geopandas(stats)
    df["year"] = year  # Add year column
    return df[["id", "year", "pdsi", "tmmn", "tmmx"]]

# Function to calculate yearly sum of precipitation
def calculate_yearly_sum_pr(year):
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"
    
    # Filter TerraClimate for the year
    yearly_pr = terraclimate.filterDate(start_date, end_date).select("pr").sum()
    
    # Reduce to sum of precipitation
    stats = yearly_pr.reduceRegions(
        collection=amc_fc,
        reducer=ee.Reducer.sum(),
        scale=4638,
        crs="EPSG:4326"
    )
    
    # Convert to pandas DataFrame
    df = geemap.ee_to_geopandas(stats)
    df["year"] = year  # Add year column
    return df[["id", "year", "pr"]]

# Process all years and combine results
years = range(2000, 2024)

# Calculate mean variables
mean_results = [calculate_yearly_mean(year) for year in years]
mean_df = pd.concat(mean_results)

# Calculate precipitation sum
sum_results = [calculate_yearly_sum_pr(year) for year in years]
sum_df = pd.concat(sum_results)

# Merge mean and sum results
final_df = pd.merge(mean_df, sum_df, on=["id", "year"])

# Clean up DataFrame for panel data
final_df = final_df.rename(columns={
    "id": "AMC_ID",
    "pdsi": "Mean_PDSI",
    "tmmn": "Mean_Min_Temp",
    "tmmx": "Mean_Max_Temp",
    "pr": "Total_Precipitation"
})

# Save to CSV
final_df.to_csv("AMC_Yearly_Panel.csv", index=False)

print("Processing completed. Results saved to 'AMC_Yearly_Panel.csv'.")
